In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(receiver_email, subject, body):
    sender_email = "shinopresents@gmail.com"
    sender_password = "insg ocuy fecc dwci"  

    try:
        # ساخت ایمیل
        message = MIMEMultipart()
        message["From"] = sender_email
        message["To"] = receiver_email
        message["Subject"] = subject
        message.attach(MIMEText(body, "plain"))

        # اتصال به Gmail SMTP
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, message.as_string())
        server.quit()

        print(f"ایمیل با موفقیت برای {receiver_email} ارسال شد ✅")
    except Exception as e:
        print(f"خطا در ارسال ایمیل به {receiver_email}: {e}")


In [ ]:
# === Project: Naqs-Amuz (Deficiency Tutor)
# Multi-module code in one file for easy copy. If you prefer real files, split by the
# markers: ===[ FILENAME ]=== and save each section accordingly.
# Python 3.10+
#
# Required libs:
#   pip install python-dotenv google-generativeai requests
#
# Files you will create alongside this script (optional but recommended):
#   .env (see template at bottom of this file)
#   responses_sample.csv (sample format at bottom)
#   kb.csv  (optional knowledge base for future use)
#
# Run:
#   python main.py
#
# Notes:
# - SQLite database file: naqsamuz.db (auto-created on first run)
# - If DB is empty, it will be seeded with 10 sample MCQs + 1 default instructor
# - LLM provider: set LLM_PROVIDER in .env to 'gemini' or 'deepseek'
# - Email: Uses SMTP with STARTTLS; configure in .env

# ===[ db_manager.py ]===
import sqlite3
import json
from typing import Optional, List, Dict, Tuple

DB_PATH = "naqsamuz.db"

SCHEMA_SQL = """
PRAGMA foreign_keys = ON;

CREATE TABLE IF NOT EXISTS questions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    question_text TEXT NOT NULL,
    options TEXT,
    correct_answer TEXT NOT NULL,
    explanatory_answer TEXT,
    tag TEXT
);

CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT,
    last_name TEXT,
    email TEXT UNIQUE NOT NULL
);

CREATE TABLE IF NOT EXISTS instructor (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS results (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id INTEGER NOT NULL,
    question_id INTEGER NOT NULL,
    is_correct INTEGER NOT NULL,
    FOREIGN KEY (student_id) REFERENCES students(id) ON DELETE CASCADE,
    FOREIGN KEY (question_id) REFERENCES questions(id) ON DELETE CASCADE
);
"""

SAMPLE_QUESTIONS: List[Dict] = [
    {
        "question_text": "Which of the following is the most likely cause of fever in a 3-year-old with sore throat and exudative tonsils?",
        "options": {"A": "Viral pharyngitis", "B": "Streptococcal pharyngitis", "C": "Allergic rhinitis", "D": "Epiglottitis"},
        "correct_answer": "B",
        "explanatory_answer": "Exudative tonsillitis with fever suggests Group A Streptococcus; confirm with RADT or throat culture.",
        "tag": "Chapter: Infectious Diseases"
    },
]

# Add more to reach 10
while len(SAMPLE_QUESTIONS) < 10:
    i = len(SAMPLE_QUESTIONS) + 1
    SAMPLE_QUESTIONS.append({
        "question_text": f"Sample question {i}: Choose the correct option.",
        "options": {"A": "Option A", "B": "Option B", "C": "Option C", "D": "Option D"},
        "correct_answer": "A" if i % 2 else "C",
        "explanatory_answer": None if i % 3 == 0 else "A brief explanation for learning.",
        "tag": "Chapter: General"
    })

DEFAULT_INSTRUCTOR = {"id": 1, "name": "Default Instructor", "email": "instructor@example.com"}


def get_conn():
    return sqlite3.connect(DB_PATH)


def setup_db() -> None:
    with get_conn() as conn:
        conn.executescript(SCHEMA_SQL)
        # Seed if empty
        cur = conn.cursor()
        cur.execute("SELECT COUNT(*) FROM questions;")
        qcount = cur.fetchone()[0]
        if qcount == 0:
            for q in SAMPLE_QUESTIONS:
                conn.execute(
                    "INSERT INTO questions (question_text, options, correct_answer, explanatory_answer, tag) VALUES (?,?,?,?,?)",
                    (q["question_text"], json.dumps(q["options"]), q["correct_answer"], q["explanatory_answer"], q["tag"]) 
                )
        cur.execute("SELECT COUNT(*) FROM instructor;")
        icount = cur.fetchone()[0]
        if icount == 0:
            conn.execute(
                "INSERT INTO instructor (id, name, email) VALUES (?,?,?)",
                (DEFAULT_INSTRUCTOR["id"], DEFAULT_INSTRUCTOR["name"], DEFAULT_INSTRUCTOR["email"]) 
            )
        conn.commit()


def list_questions() -> List[Tuple]:
    with get_conn() as conn:
        cur = conn.execute("SELECT id, question_text, options, correct_answer, IFNULL(explanatory_answer, ''), IFNULL(tag, '') FROM questions ORDER BY id;")
        return cur.fetchall()


def add_question(question_text: str, options: Dict[str, str], correct_answer: str, explanatory_answer: Optional[str], tag: Optional[str]) -> int:
    with get_conn() as conn:
        cur = conn.execute(
            "INSERT INTO questions (question_text, options, correct_answer, explanatory_answer, tag) VALUES (?,?,?,?,?)",
            (question_text, json.dumps(options), correct_answer, explanatory_answer, tag)
        )
        conn.commit()
        return cur.lastrowid


def update_explanatory_answer(question_id: int, text: str) -> None:
    with get_conn() as conn:
        conn.execute("UPDATE questions SET explanatory_answer = ? WHERE id = ?", (text, question_id))
        conn.commit()


def edit_question(question_id: int, question_text: Optional[str] = None, options: Optional[Dict[str, str]] = None,
                  correct_answer: Optional[str] = None, tag: Optional[str] = None) -> None:
    fields = []
    vals = []
    if question_text is not None:
        fields.append("question_text = ?"); vals.append(question_text)
    if options is not None:
        fields.append("options = ?"); vals.append(json.dumps(options))
    if correct_answer is not None:
        fields.append("correct_answer = ?"); vals.append(correct_answer)
    if tag is not None:
        fields.append("tag = ?"); vals.append(tag)
    if not fields:
        return
    vals.append(question_id)
    with get_conn() as conn:
        conn.execute(f"UPDATE questions SET {', '.join(fields)} WHERE id = ?", tuple(vals))
        conn.commit()


def get_or_create_student(email: str, first_name: str = "", last_name: str = "") -> int:
    with get_conn() as conn:
        cur = conn.execute("SELECT id FROM students WHERE email = ?", (email,))
        row = cur.fetchone()
        if row:
            return row[0]
        cur = conn.execute(
            "INSERT INTO students (first_name, last_name, email) VALUES (?,?,?)",
            (first_name, last_name, email)
        )
        conn.commit()
        return cur.lastrowid


def add_result(student_id: int, question_id: int, is_correct: int) -> None:
    with get_conn() as conn:
        conn.execute(
            "INSERT INTO results (student_id, question_id, is_correct) VALUES (?,?,?)",
            (student_id, question_id, is_correct)
        )
        conn.commit()


def fetch_student_mistakes(student_id: int) -> List[Tuple[int, str, Optional[str], Optional[str]]]:
    """Return list of (question_id, question_text, explanatory_answer, tag) for incorrect answers."""
    with get_conn() as conn:
        cur = conn.execute(
            """
            SELECT q.id, q.question_text, q.explanatory_answer, q.tag
            FROM results r
            JOIN questions q ON q.id = r.question_id
            WHERE r.student_id = ? AND r.is_correct = 0
            ORDER BY q.id;
            """,
            (student_id,)
        )
        return cur.fetchall()


def compute_question_stats() -> List[Tuple[int, int, int, float, Optional[str]]]:
    """Return per-question stats: (question_id, total, correct, incorrect_rate, has_expl?)"""
    with get_conn() as conn:
        cur = conn.execute(
            """
            SELECT q.id,
                   COUNT(r.id) AS total,
                   SUM(CASE WHEN r.is_correct = 1 THEN 1 ELSE 0 END) AS correct,
                   (CASE WHEN COUNT(r.id) > 0 THEN (1.0 - 1.0 * SUM(CASE WHEN r.is_correct = 1 THEN 1 ELSE 0 END) / COUNT(r.id)) ELSE 0 END) AS incorrect_rate,
                   q.explanatory_answer
            FROM questions q
            LEFT JOIN results r ON r.question_id = q.id
            GROUP BY q.id
            ORDER BY q.id;
            """
        )
        return cur.fetchall()


def get_instructor_email(default_id: int = 1) -> Optional[str]:
    with get_conn() as conn:
        cur = conn.execute("SELECT email FROM instructor WHERE id = ?", (default_id,))
        row = cur.fetchone()
        return row[0] if row else None


# ===[ llm_service.py ]===
import os
from typing import Sequence

# Provider can be 'gemini' or 'deepseek'. Configure via .env LLM_PROVIDER

class LLMService:
    def __init__(self, provider: str = "gemini"):
        self.provider = provider.lower().strip()
        self.api_key = os.getenv("LLM_API_KEY", "").strip()
        if not self.api_key:
            # We will operate in offline fallback mode (rule-based) if key missing
            pass

    def get_tag_for_question(self, question_text: str, tag_list: Sequence[str]) -> str:
        """Return one tag from tag_list that best matches question_text.
        If API not configured, do a simple keyword heuristic.
        """
        tag_list = list(tag_list)
        if not tag_list:
            return "General"

        # Offline heuristic fallback
        if not self.api_key:
            qlow = question_text.lower()
            for tag in tag_list:
                tlow = tag.lower()
                # naive keyword match
                if any(word in qlow for word in tlow.split()[:2]):
                    return tag
            return tag_list[0]

        if self.provider == "gemini":
            try:
                import google.generativeai as genai
                genai.configure(api_key=self.api_key)
                prompt = (
                    "You are a helpful classifier. Given the list of topics:\n" +
                    ", ".join([f"{t}" for t in tag_list]) +
                    "\nSelect exactly one topic that best matches the question below.\n" +
                    "Respond with the topic text only.\n\nQuestion:\n" + question_text
                )
                model = genai.GenerativeModel("gemini-1.5-flash")
                resp = model.generate_content(prompt)
                text = resp.text.strip()
                # Return closest exact tag if partial
                for tag in tag_list:
                    if tag.lower() in text.lower():
                        return tag
                # fallback to first
                return tag_list[0]
            except Exception:
                return tag_list[0]

        if self.provider == "deepseek":
            try:
                import requests
                url = os.getenv("DEEPSEEK_URL", "https://api.deepseek.com/chat/completions")
                model = os.getenv("DEEPSEEK_MODEL", "deepseek-chat")
                headers = {"Authorization": f"Bearer {self.api_key}", "Content-Type": "application/json"}
                prompt = (
                    "You are a helpful classifier. Given the list of topics:\n" +
                    ", ".join([f"{t}" for t in tag_list]) +
                    "\nSelect exactly one topic that best matches the question below.\n" +
                    "Respond with the topic text only.\n\nQuestion:\n" + question_text
                )
                data = {
                    "model": model,
                    "messages": [
                        {"role": "system", "content": "You classify text into one of provided topics."},
                        {"role": "user", "content": prompt},
                    ],
                    "temperature": 0.0,
                }
                r = requests.post(url, headers=headers, json=data, timeout=30)
                r.raise_for_status()
                out = r.json()
                text = out.get("choices", [{}])[0].get("message", {}).get("content", "").strip()
                for tag in tag_list:
                    if tag.lower() in text.lower():
                        return tag
                return tag_list[0]
            except Exception:
                return tag_list[0]

        # Unknown provider
        return tag_list[0]


# ===[ email_service.py ]===
import smtplib
from email.message import EmailMessage

SMTP_SERVER = os.getenv("SMTP_SERVER", "")
SMTP_PORT = int(os.getenv("SMTP_PORT", "587"))
SENDER_EMAIL = os.getenv("SENDER_EMAIL", "")
SENDER_PASSWORD = os.getenv("SENDER_PASSWORD", "")


import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os

# از .env بخونیم
SENDER_EMAIL = os.getenv("SENDER_EMAIL", "shinopresents@gmail.com")
SENDER_PASSWORD = os.getenv("SENDER_PASSWORD", "insg ocuy fecc dwci")

def send_email(receiver_email, subject, body):
    try:
        msg = MIMEMultipart()
        msg["From"] = SENDER_EMAIL
        msg["To"] = receiver_email
        msg["Subject"] = subject
        msg.attach(MIMEText(body, "plain"))

        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(SENDER_EMAIL, SENDER_PASSWORD)
        server.send_message(msg)
        server.quit()
        print(f"ایمیل با موفقیت به {receiver_email} ارسال شد ✅")

    except Exception as e:
        print(f"❌ خطا در ارسال ایمیل به {receiver_email}:", e)



# ===[ main.py ]===
import os
import csv
from pathlib import Path

# import from above sections (since single-file): functions already in namespace

from typing import Dict


def input_int(prompt: str, allow_empty: bool = False) -> int:
    while True:
        s = input(prompt).strip()
        if allow_empty and s == "":
            return -1
        if s.isdigit():
            return int(s)
        print("ورودی عددی معتبر نیست. دوباره تلاش کنید.")


def show_all_questions():
    rows = list_questions()
    print("\n— لیست سوالات —")
    for (qid, qtext, opts_json, correct, expl, tag) in rows:
        try:
            opts = json.loads(opts_json) if opts_json else {}
        except Exception:
            opts = {}
        print(f"\n[ID {qid}] {qtext}\n  Tag: {tag}\n  Correct: {correct}\n  Options:")
        for k, v in opts.items():
            print(f"    {k}: {v}")
        if expl:
            print(f"  Explanation: {expl[:120]}{'...' if len(expl)>120 else ''}")


def add_new_question(llm: LLMService):
    print("\n— افزودن سوال جدید —")
    qtext = input("متن سوال: ").strip()
    options: Dict[str, str] = {}
    for k in ["A", "B", "C", "D"]:
        options[k] = input(f"گزینه {k}: ").strip()
    correct = input("گزینه صحیح (A/B/C/D): ").strip().upper()
    if correct not in options:
        print("گزینه صحیح نامعتبر است. لغو شد.")
        return
    expl = input("پاسخ تشریحی (اختیاری، خالی بگذارید): ").strip()
    tag = input("Tag/Topic (خالی = تعیین خودکار با LLM): ").strip()

    if not tag:
        # choose from a simple predefined list; you can load from file as needed
        candidates = [
            "Chapter: Infectious Diseases",
            "Chapter: Cardiology",
            "Chapter: Nephrology",
            "Chapter: General",
        ]
        tag = llm.get_tag_for_question(qtext, candidates)
        print(f"Tag انتخاب‌شده: {tag}")

    qid = add_question(qtext, options, correct, (expl or None), tag)
    print(f"سوال با ID={qid} ذخیره شد.")


def add_or_update_explanatory():
    print("\n— افزودن/به‌روزرسانی پاسخ تشریحی —")
    qid = input_int("ID سوال: ")
    text = input("متن پاسخ تشریحی: ").strip()
    update_explanatory_answer(qid, text)
    print("انجام شد.")


def edit_question_cli(llm: LLMService):
    print("\n— ویرایش سوال —")
    qid = input_int("ID سوال: ")
    qtext = input("متن سوال جدید (خالی = بدون تغییر): ").strip()
    opts_change = input("تغییر گزینه‌ها؟ y/n: ").strip().lower() == 'y'
    options = None
    if opts_change:
        options = {}
        for k in ["A", "B", "C", "D"]:
            options[k] = input(f"گزینه {k}: ").strip()
    correct = input("گزینه صحیح جدید (A/B/C/D یا خالی): ").strip().upper()
    if correct and correct not in ["A","B","C","D"]:
        print("گزینه صحیح نامعتبر؛ بخش اصلاح نشد.")
        correct = None
    tag = input("Tag جدید (خالی = بدون تغییر، 'auto' = تعیین با LLM): ").strip()
    if tag == 'auto':
        # we need text to classify; prefer new text or fetch existing
        rows = [r for r in list_questions() if r[0] == qid]
        base_qtext = qtext or (rows[0][1] if rows else "")
        candidates = ["Chapter: Infectious Diseases","Chapter: Cardiology","Chapter: Nephrology","Chapter: General"]
        tag = llm.get_tag_for_question(base_qtext, candidates)
    elif tag == "":
        tag = None

    edit_question(qid, question_text=(qtext or None), options=options, correct_answer=(correct or None), tag=tag)
    print("ویرایش انجام شد.")


def analyze_and_send_feedback():
    print("\n— آنالیز نتایج و ارسال بازخورد —")
    path = input("مسیر فایل CSV نتایج (ستون‌ها: student_email,question_id,is_correct): ").strip()
    instr_email = input("ایمیل استاد (خالی = مقدار پیش‌فرض دیتابیس): ").strip()
    if not instr_email:
        instr_email = get_instructor_email() or "instructor@example.com"

    if not Path(path).exists():
        print("فایل پیدا نشد.")
        return

    # ingest
    per_student_rows: Dict[str, list] = {}
    with open(path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        required = {"student_email", "question_id", "is_correct"}
        if not required.issubset(set(reader.fieldnames or [])):
            print("ستون‌های فایل معتبر نیست.")
            return
        for row in reader:
            email = row["student_email"].strip().lower()
            try:
                qid = int(row["question_id"].strip())
                is_correct = 1 if str(row["is_correct"]).strip() in ("1","true","True","YES","yes") else 0
            except Exception:
                print(f"سطر نامعتبر رد شد: {row}")
                continue
            sid = get_or_create_student(email)
            add_result(sid, qid, is_correct)
            per_student_rows.setdefault(email, []).append((qid, is_correct))

    # student feedback emails
    for email in per_student_rows.keys():
        sid = get_or_create_student(email)
        mistakes = fetch_student_mistakes(sid)
        if not mistakes:
            body = (
                "سلام\n\n"
                "نتایج شما بررسی شد. در این آزمون پاسخ‌های شما صحیح بوده یا اشتباه معنی‌داری ثبت نشده است.\n"
                "با آرزوی موفقیت."
            )
        else:
            # group by tag and list short hints
            tag_map: Dict[str, list] = {}
            for qid, qtext, expl, tag in mistakes:
                key = tag or "General"
                tag_map.setdefault(key, []).append((qid, qtext, expl))
            parts = ["سلام", "\nنتایج آزمون شما بررسی شد. در مباحث زیر نیاز به مرور بیشتری دارید:"]
            for tag, items in tag_map.items():
                parts.append(f"\n— {tag} —")
                for (qid, qtext, expl) in items:
                    if expl and expl.strip():
                        parts.append(f"• نکتهٔ کلیدی: {expl.strip()}")
                    else:
                        parts.append("• برای این مورد، پاسخ تشریحی به‌زودی افزوده خواهد شد.")
            parts.append("\nپیشنهاد: منابع اصلی همان فصل/بخش مذکور در رفرنس مربوطه است. موفق باشید.")
            body = "\n".join(parts)
        send_email(email, "گزارش بازخورد شخصی آزمون", body)

    # instructor report
    stats = compute_question_stats()
    needs_expl = [
        (qid, total, correct, incorrect_rate)
        for (qid, total, correct, incorrect_rate, expl) in stats
        if total > 0 and incorrect_rate > 0.5 and (expl is None or str(expl).strip() == "")
    ]
    lines = [
        "سلام استاد محترم",
        "گزارش خلاصه آزمون تهیه شد.",
        "\nسوالاتی که بیش از ۵۰٪ پاسخ اشتباه داشته‌اند و فاقد پاسخ تشریحی هستند:",
    ]
    if not needs_expl:
        lines.append("— موردی یافت نشد.")
    else:
        for (qid, total, correct, ir) in needs_expl:
            pc = round(ir * 100, 1)
            lines.append(f"• سوال ID {qid}: درصد خطا {pc}% از {total} پاسخ")
    lines.append("\nبا احترام")
    send_email(instr_email, "گزارش سوالات نیازمند پاسخ تشریحی", "\n".join(lines))

    # Also print overall stats to console
    print("\n— آمار سوالات —")
    for (qid, total, correct, incorrect_rate, expl) in stats:
        pc = round(incorrect_rate * 100, 1)
        print(f"ID {qid}: total={total}, correct={correct}, incorrect%={pc}, explained={'Yes' if (expl and str(expl).strip()) else 'No'}")


def main():
    setup_db()
    llm = LLMService(provider=os.getenv("LLM_PROVIDER", "gemini"))

    MENU = {
        "1": ("Show all questions", show_all_questions),
        "2": ("Add a new question", lambda: add_new_question(llm)),
        "3": ("Add/Update an explanatory answer for a question", add_or_update_explanatory),
        "4": ("Edit a question", lambda: edit_question_cli(llm)),
        "5": ("Analyze exam results and send feedback", analyze_and_send_feedback),
        "0": ("Exit", None),
    }

    while True:
        print("\n==== Naqs-Amuz ====")
        for k, (title, _) in MENU.items():
            print(f"{k}. {title}")
        choice = input("گزینه را وارد کنید: ").strip()
        if choice == "0":
            print("خروج...")
            break
        func = MENU.get(choice, (None, None))[1]
        if func is None:
            print("گزینه نامعتبر.")
            continue
        try:
            func()
        except Exception as e:
            print(f"خطا: {e}")


if __name__ == "__main__":
    main()

# ===[ .env TEMPLATE ]===
# Copy the block below into a file named .env in project root and fill values:
"""
LLM_PROVIDER=gemini
LLM_API_KEY=your_api_key_here

# If using DeepSeek instead:
# LLM_PROVIDER=deepseek
# LLM_API_KEY=your_deepseek_api_key
# DEEPSEEK_URL=https://api.deepseek.com/chat/completions
# DEEPSEEK_MODEL=deepseek-chat

SENDER_EMAIL=your_email@example.com
SENDER_PASSWORD=your_password_or_app_password
SMTP_SERVER=smtp.example.com
SMTP_PORT=587
"""

# ===[ responses_sample.csv ]===
# Save as responses_sample.csv (UTF-8):
"""
student_email,question_id,is_correct
ali@example.com,1,1
ali@example.com,2,0
sara@example.com,2,0
sara@example.com,3,1
"""



==== Naqs-Amuz ====
1. Show all questions
2. Add a new question
3. Add/Update an explanatory answer for a question
4. Edit a question
5. Analyze exam results and send feedback
0. Exit


گزینه را وارد کنید:  5



— آنالیز نتایج و ارسال بازخورد —


مسیر فایل CSV نتایج (ستون‌ها: student_email,question_id,is_correct):  C:\Users\admin\Downloads\sample_exam_results.csv
ایمیل استاد (خالی = مقدار پیش‌فرض دیتابیس):  Sepehr1tak22@gmail.com


ایمیل با موفقیت به sepehr1tak22@gmail.com ارسال شد ✅
ایمیل با موفقیت به Sepehr1tak22@gmail.com ارسال شد ✅

— آمار سوالات —
ID 1: total=3, correct=3, incorrect%=0.0, explained=Yes
ID 2: total=3, correct=0, incorrect%=100.0, explained=Yes
ID 3: total=3, correct=3, incorrect%=0.0, explained=No
ID 4: total=3, correct=3, incorrect%=0.0, explained=Yes
ID 5: total=3, correct=0, incorrect%=100.0, explained=Yes
ID 6: total=3, correct=0, incorrect%=100.0, explained=No
ID 7: total=3, correct=3, incorrect%=0.0, explained=Yes
ID 8: total=3, correct=3, incorrect%=0.0, explained=Yes
ID 9: total=3, correct=0, incorrect%=100.0, explained=No
ID 10: total=3, correct=3, incorrect%=0.0, explained=Yes
ID 11: total=3, correct=0, incorrect%=100.0, explained=Yes

==== Naqs-Amuz ====
1. Show all questions
2. Add a new question
3. Add/Update an explanatory answer for a question
4. Edit a question
5. Analyze exam results and send feedback
0. Exit
